In [1]:
"""
Technical Indicators for Stock Market 
"""

'\nTechnical Indicators for Stock Market \n'

In [5]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from datetime import date

import yfinance as yf

In [6]:
START = '2018-01-01'
TODAY = date.today().strftime("%Y-%m-%d")

selected_stocks = "AAPL"

def load_data(ticker):
    data = yf.download(ticker, START, TODAY)
    data.reset_index(inplace=True)
    return data

data = load_data(selected_stocks)
data.head()

[*********************100%***********************]  1 of 1 completed


,Date,Open,High,Low,Close,Adj Close,Volume
0,2018-01-02,42.540001,43.075001,42.314999,43.064999,41.248272,102223600
1,2018-01-03,43.132500,43.637501,42.990002,43.057499,41.241089,118071600
2,2018-01-04,43.134998,43.367500,43.020000,43.257500,41.432659,89738400
3,2018-01-05,43.360001,43.842499,43.262501,43.750000,41.904385,94640000
4,2018-01-08,43.587502,43.902500,43.482498,43.587502,41.748737,82271200


In [8]:
df = data.set_index(pd.DatetimeIndex(data['Date']).values)
df

,Date,Open,High,Low,Close,Adj Close,Volume
2018-01-02,2018-01-02,42.540001,43.075001,42.314999,43.064999,41.248272,102223600
2018-01-03,2018-01-03,43.132500,43.637501,42.990002,43.057499,41.241089,118071600
2018-01-04,2018-01-04,43.134998,43.367500,43.020000,43.257500,41.432659,89738400
2018-01-05,2018-01-05,43.360001,43.842499,43.262501,43.750000,41.904385,94640000
2018-01-08,2018-01-08,43.587502,43.902500,43.482498,43.587502,41.748737,82271200
...,...,...,...,...,...,...,...
2021-08-02,2021-08-02,146.360001,146.949997,145.250000,145.520004,145.302307,62880000
2021-08-03,2021-08-03,145.809998,148.039993,145.179993,147.360001,147.139542,64786600
2021-08-04,2021-08-04,147.270004,147.789993,146.279999,146.949997,146.730164,56368300
2021-08-05,2021-08-05,146.979996,147.839996,146.169998,147.059998,146.839996,46397700


In [9]:
# Simple Moving Average (SMA)
def generate_SMA(data, period = 30, column = 'Close'):
    return data[column].rolling(window = period).mean()

# Exponential Moving Average (EMA)
def generate_EMA(data, period = 20, column = 'Close'):
    return data[column].ewm(span = period, adjust = False).mean()

# Moving Average Convergence / Divergence (MACD)
def generate_MACD(data, period_long = 26, period_short = 12, period_signal = 9, column = 'Close'):
    
    # Short term EMA
    short_EMA = generate_EMA(data, period_short, column)

    # Long term EMA
    long_EMA = generate_EMA(data, period_long, column)

    data['MACD'] = short_EMA - long_EMA

    data['Signal_Line'] = generate_EMA(data, period_signal, "MACD")

    return data

In [ ]:
# Relative Strength Index (RSI) 
def generate_RSI(data, period = 14, column = 'Close'):
    delta = data[column].diff(1)
    delta = delta[1:]
    
    up = delta.copy()
    down = delta.copy()

    up[ up < 0] = 0
    down[ down > 0] = 0

    data['up'] = up
    data['down'] = down

    AVG_gain = generate_SMA(data, period, 'up')
    AVG_loss = generate_SMA(data, period, 'down')

    RSI = 100.0 - (100 / (1 + (AVG_gain / AVG_loss)))

    data['RSI'] = RSI

    return data
